<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/tensorflow_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%env TF_CPP_MIN_LOG_LEVEL=1  # Disable logs on TF import

## Installation

The CLI tool is installed with `tensorflow-datasets` (or `tfds-nightly`).

In [ ]:
!pip install -q tfds-nightly
!tfds --version

For the list of all CLI commands:

In [ ]:
!tfds --help

## `tfds new`: Implementing a new Dataset

This command will help you kickstart writing your new Python dataset by creating
a `<dataset_name>/` directory containing default implementation files.

Usage:

In [ ]:
!tfds new my_dataset

Will create:

In [ ]:
ls -1 my_dataset/

See our [writing dataset guide](https://www.tensorflow.org/datasets/add_dataset)
for more info.

Available options:

In [ ]:
!tfds new --help

## `tfds build`: Download and prepare a dataset

Use `tfds build <my_dataset>` to generate a new dataset. `<my_dataset>` can be:

* A path to `dataset/` folder or `dataset.py` file (empty for current directory):
  * `tfds build datasets/my_dataset/`
  * `cd datasets/my_dataset/ && tfds build`
  * `cd datasets/my_dataset/ && tfds build my_dataset`
  * `cd datasets/my_dataset/ && tfds build my_dataset.py`

* A registered dataset:

  * `tfds build mnist`
  * `tfds build my_dataset --imports my_project.datasets`

Note: `tfds build` has useful flags to help prototyping and debuging. See the `Debug & tests:` section bellow.

Available options:

In [ ]:
!tfds build --help